# **Damage Assessment**

<span style="font-family: times, Optima; font-size:11pt; color:black;">
    
> **Buildings:** 
Each building has associated a replacement cost expressed in 2010 US dollars (USD).  

> **Roads  and Special Infrastructures:**
None of these layers have a replacement cost associated as an attribute, therefore it has been obtained from the PCRAFI report (The World Bank 2013). The roads have also been classified based on the hazard value in transitable (water depth<0.3) or not transitable (water depth>0.3).

> **Crops:**
 The typology has been divided into three main classes, tree crops, root crops, and intercrops, since the effects of the wind and flooding are different for each category.

> **Population:**
No economic damage is assessed to the population, however according to Cox et al. (2010) different safety threads have been defined based on water depth (similar with wind) what can be more useful on a forecasted event.
    
<span>

In [ ]:
import os
import os.path as op
import sys
import time

# basics
import glob
import xarray as xr
import pandas as pd
#import numpy as np

# raster tools
import geopandas as gpd
import json
import rioxarray
import shutil

sys.path.insert(0, op.join(os.getcwd(), '..',  '..', 'lib'))

# kepler 
from config import *
from keplergl import KeplerGl

In [ ]:
# hazard path
p_data = op.join(os.getcwd(), '..', 'data')
project_path = op.join(p_data, 'lisflood', 'projects')

# executable path
exec_path = op.join(os.getcwd(), '..', '..', 'lib', 'riskscape', 'bin', 'riskscape')

# riskscape folder
p_riskscape = op.join(p_data, 'riskscape', 'projects')

## **Run riskscape model**

In [ ]:
models = ['landcrops_dam', 'especials', 'Build_dam', 'dam_people', 'dam_roads']

In [ ]:
for event in os.listdir(project_path):
    
    print('\rRunning Model ' + event)
    
    if not op.exists(op.join(p_riskscape, event)):
        os.makedirs(op.join(p_riskscape, event))
    shutil.copy(op.join(p_riskscape, 'bookmarks.ini'), op.join(p_riskscape, event, 'bookmarks.ini'))
    shutil.copy(op.join(p_riskscape, 'project.ini'), op.join(p_riskscape, event, 'project.ini'))
    shutil.copy(op.join(project_path, event, 'flooding_raster.tif'), op.join(p_riskscape, event, 'flooding_raster.tif'))
    
    os.chdir(op.join(p_riskscape, event))
    
    start_time = time.time()
    
    for model in models:

        print('\rRunning Model : {0}'.format(model))
        path_output = op.join(p_riskscape, event, model)
        list = [exec_path, ' model run ', model, ' --output ', path_output, ' -r']
        to_run = ''.join(list)
        stream = os.popen(to_run)
        output = stream.read()

    print("--- %Total seconds --- {0:.1f}".format(time.time() - start_time))
    

## **Plot Damage over assets**

In [ ]:
event = 'case0'

**Impact over assets - Output RiskScape**

In [ ]:
land_dmg = gpd.read_file(op.join(p_riskscape, event, 'landcrops_dam','event-impact.shp'))
build_dmg = gpd.read_file(op.join(p_riskscape, event, 'Build_dam','event-impact.shp'))
spec_dmg = gpd.read_file(op.join(p_riskscape, event, 'especials','event-impact.shp'))
people_dmg = gpd.read_file(op.join(p_riskscape, event, 'dam_people','event-impact.shp'))
roads_dmg = gpd.read_file(op.join(p_riskscape, event, 'dam_roads','event-impact.shp'))

**Convert X, Y coordinates to Lon, Lat**

In [ ]:
build_dmg = build_dmg.to_crs(epsg=4326)
people_dmg = people_dmg.to_crs(epsg=4326)
spec_dmg = spec_dmg.to_crs(epsg=4326)
land_dmg = land_dmg.to_crs(epsg=4326)
roads_dmg = roads_dmg.to_crs(epsg=4326)

In [ ]:
build_dmg = build_dmg[['Occ', 'Exposed', 'Loss', 'geometry']]
land_dmg = land_dmg[['CLASS_NAME', 'Loss', 'geometry']]
spec_dmg = spec_dmg[['TYPE', 'Loss', 'geometry']]
people_dmg = people_dmg[['Safety_Thr', 'Residents_', 'geometry']]
roads_dmg = roads_dmg[['CLASS', 'Impassab_1', 'Loss', 'geometry']]

In [ ]:
# Load Hazard - winds SWATH
a_file = open(op.join(p_riskscape, '..', 'config_files', 'config_damage.json'), "rb")
config_damage = json.load(a_file)
a_file.close()

map_1 = KeplerGl(height=700, show_docs=False)

# Load Damage - Buildings, Land and Roads
map_1.add_data(data=build_dmg, name='Buildings Damage')
map_1.add_data(data=land_dmg, name='Land Damage')
map_1.add_data(data=spec_dmg, name='Specials Damage')
map_1.add_data(data=roads_dmg, name='Roads Damage')

map_1